In [31]:
import math
import numpy as np
from numba import cuda, jit, int32
from datetime import datetime
import time


In [32]:
def get_cur_time():
  return time.time()*1000

In [33]:
@cuda.jit
def gpu_vec_sum(a, b, c):
  idx = cuda.blockIdx.x * cuda.blockDim.x + cuda.threadIdx.x
  sum = 0
  if idx < len(a):
    
    sum += a[idx]+b[idx]
    cuda.atomic.add(c, 0, sum)
    
def do_sum_with_gpu(a, b, c, col_):
    gpu_a = cuda.to_device(a)
    gpu_b = cuda.to_device(b)
    gpu_c = cuda.to_device(c)
    start = get_cur_time()
    gpu_vec_sum[col_, TPB](gpu_a, gpu_b, gpu_c)
    itog_time = get_cur_time()-start
    return gpu_c.copy_to_host()[0], itog_time

In [34]:

TPB = 32
arr_len = 1000

for k_ in range(4): 
  
  a = []
  for i in range(arr_len):
    a.append(i)
  b = []
  for i in range(int((arr_len)/2)):
    b.append(a[-1])
    a.pop()

  col_ = (math.ceil((len(a))/TPB))  
  c1 = np.zeros(1, dtype = np.int64)
  cpu_start = get_cur_time()
  cpu_sum = sum(a) + sum(b)
  cpu_time = get_cur_time()-cpu_start

  gpu_sum, gpu_time = do_sum_with_gpu(a, b, c1, col_)
  print("CPU: %s elements" % arr_len)
  print(cpu_sum, cpu_time)
  print("GPU: %s elements" % arr_len)
  print(gpu_sum, gpu_time)
  arr_len = arr_len*10

/usr/local/lib/python3.7/dist-packages/numba/cuda/dispatcher.py:488: NumbaPerformanceWarning: Grid size 16 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))


CPU: 1000 elements
499500 0.0126953125
GPU: 1000 elements
499500 128.359375
CPU: 10000 elements
49995000 0.073974609375
GPU: 10000 elements
49995000 0.713134765625
CPU: 100000 elements
4999950000 0.62548828125
GPU: 100000 elements
4999950000 0.835205078125
CPU: 1000000 elements
499999500000 7.291748046875
GPU: 1000000 elements
499999500000 0.18408203125
